<a href="https://colab.research.google.com/github/Frederick-Teye/ML-models/blob/main/Copy_of_last_last.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import pandas as pd
ziya_data = pd.read_csv('/content/drive/MyDrive/data-for-datamining-class/kaggle/cleaned_ziya_data.csv')
ziya_data.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,hbA1c_level,blood_glucose_level,diabetes
0,Female,56.0,0,0,No Info,37.23,6.5,80,0
1,Male,29.0,0,0,never,27.32,3.5,160,0
2,Female,26.0,0,0,No Info,27.32,4.0,145,0
3,Male,50.0,0,0,current,27.32,5.0,145,0
4,Female,56.0,0,0,never,38.48,5.7,155,1


In [24]:
import os
print(os.listdir('.'))

['.config', 'drive', 'sample_data']


Please upload the `cleaned_ziya_data.csv` file into your Colab environment. You can do this by clicking the folder icon on the left sidebar, then clicking the 'Upload to session storage' icon, and selecting your file. After uploading, run the next cell to load the data.

In [25]:
ziya_data.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,hbA1c_level,blood_glucose_level,diabetes
0,Female,56.0,0,0,No Info,37.23,6.5,80,0
1,Male,29.0,0,0,never,27.32,3.5,160,0
2,Female,26.0,0,0,No Info,27.32,4.0,145,0
3,Male,50.0,0,0,current,27.32,5.0,145,0
4,Female,56.0,0,0,never,38.48,5.7,155,1


In [26]:
ziya_data.groupby(['gender', 'hypertension', 'heart_disease']).diabetes.count()

gender  hypertension  heart_disease
Female  0             0                7620
                      1                 439
        1             0                1207
                      1                 184
Male    0             0                5674
                      1                 673
        1             0                 979
                      1                 224
Name: diabetes, dtype: int64

In [27]:
ziya_data.diabetes.value_counts()

,count
diabetes,
0,8500
1,8500


In [28]:
# Balanced code

In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, classification_report)

# Load data
X = ziya_data.drop(columns=['diabetes'])  # Feature matrix
y = ziya_data['diabetes']  # Target variable

# Train-test split (stratify to maintain balance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Feature selection
num_features = ['age', 'hypertension', 'heart_disease', 'bmi',
                'hbA1c_level', 'blood_glucose_level']
cat_features = ['gender', 'smoking_history']

# Categorical Encoding (One-Hot Encoding)
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), cat_features)
    ],
    remainder='passthrough'
)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# XGBoost Classifier (No class weighting for balanced data)
xgb_model = XGBClassifier(
    n_estimators=900,            # Number of boosting rounds
    learning_rate=0.01,          # Learning rate (lower = better generalization)
    early_stopping_rounds=10,    # Stop training if validation loss doesn't improve
    eval_metric=['aucpr', 'logloss'],  # Evaluation metrics
    random_state=42,             # Random seed for reproducibility
    max_depth=6,                 # Tree depth (higher = more complex model)
    min_child_weight=3,          # Regularization parameter
    subsample=0.8,               # Fraction of samples used per boosting round
    colsample_bytree=0.9         # Fraction of features used per tree
)

# Training with validation set
eval_set = [(X_train_preprocessed, y_train), (X_test_preprocessed, y_test)]
xgb_model.fit(X_train_preprocessed, y_train, eval_set=eval_set, verbose=False)

# Get probabilities
y_pred_proba = xgb_model.predict_proba(X_test_preprocessed)[:, 1]

# Decision Threshold
threshold = 0.5  # Standard for balanced data
y_pred = (y_pred_proba >= threshold).astype(int)

# Model Evaluation
print(f"\nEvaluation at Threshold = {threshold:.2f}")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Store metrics for visualization
recall_model0 = recall_score(y_test, y_pred)
precision_model0 = precision_score(y_test, y_pred)
f1_model0 = f1_score(y_test, y_pred)
accuracy_model0 = accuracy_score(y_test, y_pred)
roc_auc_model0 = roc_auc_score(y_test, y_pred_proba)



Evaluation at Threshold = 0.50
Accuracy: 0.9091
Precision: 0.9009
Recall: 0.9194
F1-Score: 0.9100
ROC-AUC: 0.9766

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.90      0.91      1700
           1       0.90      0.92      0.91      1700

    accuracy                           0.91      3400
   macro avg       0.91      0.91      0.91      3400
weighted avg       0.91      0.91      0.91      3400



In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, classification_report)

# Load data
X = ziya_data.drop(columns=['diabetes'])  # Feature matrix
y = ziya_data['diabetes']  # Target variable

# Train-test split (stratify to maintain balance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Feature selection
num_features = ['age', 'hypertension', 'heart_disease', 'bmi',
                'hbA1c_level', 'blood_glucose_level']
cat_features = ['gender', 'smoking_history']

# Categorical Encoding (One-Hot Encoding)
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), cat_features)
    ],
    remainder='passthrough'
)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# XGBoost Classifier (No class weighting for balanced data)
xgb_model = XGBClassifier(
    n_estimators=1000,            # Number of boosting rounds
    learning_rate=0.01,          # Learning rate (lower = better generalization)
    early_stopping_rounds=10,    # Stop training if validation loss doesn't improve
    eval_metric=['aucpr', 'logloss'],  # Evaluation metrics
    random_state=42,             # Random seed for reproducibility
    max_depth=6,                 # Tree depth (higher = more complex model)
    min_child_weight=3,          # Regularization parameter
    subsample=0.8,               # Fraction of samples used per boosting round
    colsample_bytree=0.9         # Fraction of features used per tree
)

# Training with validation set
eval_set = [(X_train_preprocessed, y_train), (X_test_preprocessed, y_test)]
xgb_model.fit(X_train_preprocessed, y_train, eval_set=eval_set, verbose=False)

# Get probabilities
y_pred_proba = xgb_model.predict_proba(X_test_preprocessed)[:, 1]

# Decision Threshold
threshold = 0.5  # Standard for balanced data
y_pred = (y_pred_proba >= threshold).astype(int)

# Model Evaluation
print(f"\nEvaluation at Threshold = {threshold:.2f}")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Store metrics for visualization
recall_model0 = recall_score(y_test, y_pred)
precision_model0 = precision_score(y_test, y_pred)
f1_model0 = f1_score(y_test, y_pred)
accuracy_model0 = accuracy_score(y_test, y_pred)
roc_auc_model0 = roc_auc_score(y_test, y_pred_proba)



Evaluation at Threshold = 0.50
Accuracy: 0.9091
Precision: 0.9009
Recall: 0.9194
F1-Score: 0.9100
ROC-AUC: 0.9766

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.90      0.91      1700
           1       0.90      0.92      0.91      1700

    accuracy                           0.91      3400
   macro avg       0.91      0.91      0.91      3400
weighted avg       0.91      0.91      0.91      3400



In [31]:
# Hyperparameter tuning using Optuna

In [32]:
import sklearn
import xgboost
print(f"scikit-learn version: {sklearn.__version__}")
print(f"xgboost version: {xgboost.__version__}")

scikit-learn version: 1.6.1
xgboost version: 3.1.3


# hpelm

In [33]:
!pip install hpelm
import hpelm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

# Load data
X = ziya_data.drop(columns=['diabetes'])
y = ziya_data['diabetes']

# Convert target variable to one-hot encoding
ohe = OneHotEncoder(drop=None, sparse_output=False)
y_onehot = ohe.fit_transform(y.values.reshape(-1, 1))

# Split data with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42, stratify=y
)

# Define available features
available_features = X_train.columns.tolist()

# Preprocessing pipeline
num_features = ['age', 'hypertension', 'heart_disease', 'bmi', 'hbA1c_level', 'blood_glucose_level']
cat_features = ['gender', 'smoking_history']

# Remove missing features
num_features = [col for col in num_features if col in available_features]
cat_features = [col for col in cat_features if col in available_features]

# Define preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), cat_features)
    ]
)

# Apply transformations
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)


print(f' X_train_preprocessed.shape[1]:\n  {X_train_preprocessed.shape[1]}')
print(f'y_train.shape[1]:\n {y_train.shape[1]}')

# Define the ELM model
num_neurons = 3000  # Adjust as needed
elm_model = hpelm.ELM(X_train_preprocessed.shape[1], y_train.shape[1])
elm_model.add_neurons(num_neurons, "sigm")  # Using sigmoid activation

# Train the model
elm_model.train(X_train_preprocessed, y_train, "c")  # Classification mode

# Predict probabilities
y_pred_proba = elm_model.predict(X_test_preprocessed)

# Apply threshold for classification
y_pred = np.argmax(y_pred_proba, axis=1)

# Convert y_test back to single-column labels
y_test_labels = np.argmax(y_test, axis=1)

# Evaluate performance
print("\nEvaluation Metrics")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test_labels, y_pred):.4f}")
print(f"Precision: {precision_score(y_test_labels, y_pred):.4f}")
print(f"Recall: {recall_score(y_test_labels, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test_labels, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test_labels, y_pred_proba[:, 1]):.4f}")
print("\nClassification Report:")
print(classification_report(y_test_labels, y_pred))

 X_train_preprocessed.shape[1]:
  12
y_train.shape[1]:
 2

Evaluation Metrics
Accuracy: 0.8682
Precision: 0.8687
Recall: 0.8676
F1-Score: 0.8682
ROC-AUC: 0.9387

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.87      0.87      1700
           1       0.87      0.87      0.87      1700

    accuracy                           0.87      3400
   macro avg       0.87      0.87      0.87      3400
weighted avg       0.87      0.87      0.87      3400



In [34]:
import hpelm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

# Load data
X = ziya_data.drop(columns=['diabetes'])
y = ziya_data['diabetes']

# Convert target variable to one-hot encoding
ohe = OneHotEncoder(drop=None, sparse_output=False)
y_onehot = ohe.fit_transform(y.values.reshape(-1, 1))

# Split data with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42, stratify=y
)

# Define available features
available_features = X_train.columns.tolist()

# Preprocessing pipeline
num_features = ['age', 'hypertension', 'heart_disease', 'bmi', 'hbA1c_level', 'blood_glucose_level']
cat_features = ['gender', 'smoking_history']

# Remove missing features
num_features = [col for col in num_features if col in available_features]
cat_features = [col for col in cat_features if col in available_features]

# Define preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), cat_features)
    ]
)

# Apply transformations
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Define the ELM model
num_neurons = 3000  # Adjust as needed
elm_model = hpelm.ELM(X_train_preprocessed.shape[1], y_train.shape[1])
elm_model.add_neurons(num_neurons, "sigm")  # Using sigmoid activation

# Train the model
elm_model.train(X_train_preprocessed, y_train, "c")  # Classification mode

# Predict probabilities
y_pred_proba = elm_model.predict(X_test_preprocessed)

# Apply threshold for classification
y_pred = np.argmax(y_pred_proba, axis=1)

# Convert y_test back to single-column labels
y_test_labels = np.argmax(y_test, axis=1)

# Evaluate performance
print("\nEvaluation Metrics")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test_labels, y_pred):.4f}")
print(f"Precision: {precision_score(y_test_labels, y_pred):.4f}")
print(f"Recall: {recall_score(y_test_labels, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test_labels, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test_labels, y_pred_proba[:, 1]):.4f}")
print("\nClassification Report:")
print(classification_report(y_test_labels, y_pred))



Evaluation Metrics
Accuracy: 0.8729
Precision: 0.8760
Recall: 0.8688
F1-Score: 0.8724
ROC-AUC: 0.9422

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.88      0.87      1700
           1       0.88      0.87      0.87      1700

    accuracy                           0.87      3400
   macro avg       0.87      0.87      0.87      3400
weighted avg       0.87      0.87      0.87      3400



In [35]:
import hpelm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, classification_report, precision_recall_curve)
from sklearn.utils.class_weight import compute_class_weight

# Load data (same as HPELM)
X = ziya_data.drop(columns=['diabetes'])
y = ziya_data['diabetes']

# Feature Engineering: Create BMI categories
X['bmi_category'] = pd.cut(X['bmi'], bins=[0, 18.5, 25, 30, np.inf], labels=['Underweight', 'Normal', 'Overweight', 'Obese'])

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Convert target variable to one-hot encoding
ohe = OneHotEncoder(drop=None, sparse_output=False)
y_onehot = ohe.fit_transform(y.values.reshape(-1, 1))

# Split data with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42, stratify=y
)

# Define available features
available_features = X_train.columns.tolist()

# Preprocessing pipeline
num_features = ['age', 'hypertension', 'heart_disease',
       'bmi', 'hbA1c_level', 'blood_glucose_level']
cat_features = ['gender', 'smoking_history', 'bmi_category']  # Added bmi_category

# Remove missing features
num_features = [col for col in num_features if col in available_features]
cat_features = [col for col in cat_features if col in available_features]

# Define preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), num_features),  # Changed to RobustScaler
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features)  # Removed drop='first'
    ]
)

# Apply transformations
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Dynamically adjust the number of neurons
num_neurons = min(5000, X_train_preprocessed.shape[0] // 2)

# Define the ELM model
elm_model = hpelm.ELM(X_train_preprocessed.shape[1], y_train.shape[1])
elm_model.add_neurons(num_neurons, "sigm")  # Using sigmoid activation

# Train the model with class weights
elm_model.train(X_train_preprocessed, y_train, "c", W=class_weight_dict)  # Classification mode

# Predict probabilities
y_pred_proba = elm_model.predict(X_test_preprocessed)

# Determine best threshold using Precision-Recall Curve
precisions, recalls, thresholds = precision_recall_curve(np.argmax(y_test, axis=1), y_pred_proba[:, 1])
best_threshold = thresholds[np.argmax(precisions + recalls)]

# Apply optimized threshold
y_pred = (y_pred_proba[:, 1] >= best_threshold).astype(int)

# Convert y_test back to single-column labels
y_test_labels = np.argmax(y_test, axis=1)

# Evaluate performance
print("\nEvaluation Metrics")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test_labels, y_pred):.4f}")
print(f"Precision: {precision_score(y_test_labels, y_pred):.4f}")
print(f"Recall: {recall_score(y_test_labels, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test_labels, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test_labels, y_pred_proba[:, 1]):.4f}")
print("\nClassification Report:")
print(classification_report(y_test_labels, y_pred))



Evaluation Metrics
Accuracy: 0.8303
Precision: 0.7985
Recall: 0.8835
F1-Score: 0.8389
ROC-AUC: 0.8860

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.78      0.82      1700
           1       0.80      0.88      0.84      1700

    accuracy                           0.83      3400
   macro avg       0.83      0.83      0.83      3400
weighted avg       0.83      0.83      0.83      3400



In [36]:
import hpelm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, classification_report, precision_recall_curve)
from sklearn.utils.class_weight import compute_class_weight

# Load data (same as HPELM)
X = ziya_data.drop(columns=['diabetes'])
y = ziya_data['diabetes']

# Feature Engineering: Create BMI categories
X['bmi_category'] = pd.cut(X['bmi'], bins=[0, 18.5, 25, 30, np.inf], labels=['Underweight', 'Normal', 'Overweight', 'Obese'])

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Convert target variable to one-hot encoding
ohe = OneHotEncoder(drop=None, sparse_output=False)
y_onehot = ohe.fit_transform(y.values.reshape(-1, 1))

# Split data with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42, stratify=y
)

# Define available features
available_features = X_train.columns.tolist()

# Preprocessing pipeline
num_features = ['age', 'hypertension', 'heart_disease',
       'bmi', 'hbA1c_level', 'blood_glucose_level']
cat_features = ['gender', 'smoking_history', 'bmi_category']  # Added bmi_category

# Remove missing features
num_features = [col for col in num_features if col in available_features]
cat_features = [col for col in cat_features if col in available_features]

# Define preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), num_features),  # Changed to RobustScaler
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features)  # Removed drop='first'
    ]
)

# Apply transformations
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Dynamically adjust the number of neurons
num_neurons = min(5000, X_train_preprocessed.shape[0] // 2)

# Define the ELM model
elm_model = hpelm.ELM(X_train_preprocessed.shape[1], y_train.shape[1])
elm_model.add_neurons(num_neurons, "sigm")  # Using sigmoid activation

# Train the model with class weights
elm_model.train(X_train_preprocessed, y_train, "c", W=class_weight_dict)  # Classification mode

# Predict probabilities
y_pred_proba = elm_model.predict(X_test_preprocessed)

# Determine best threshold using Precision-Recall Curve
precisions, recalls, thresholds = precision_recall_curve(np.argmax(y_test, axis=1), y_pred_proba[:, 1])
best_threshold = thresholds[np.argmax(precisions + recalls)]

# Apply optimized threshold
y_pred = (y_pred_proba[:, 1] >= best_threshold).astype(int)

# Convert y_test back to single-column labels
y_test_labels = np.argmax(y_test, axis=1)

# Evaluate performance
print("\nEvaluation Metrics")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test_labels, y_pred):.4f}")
print(f"Precision: {precision_score(y_test_labels, y_pred):.4f}")
print(f"Recall: {recall_score(y_test_labels, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test_labels, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test_labels, y_pred_proba[:, 1]):.4f}")
print("\nClassification Report:")
print(classification_report(y_test_labels, y_pred))



Evaluation Metrics
Accuracy: 0.8382
Precision: 0.8142
Recall: 0.8765
F1-Score: 0.8442
ROC-AUC: 0.8953

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.80      0.83      1700
           1       0.81      0.88      0.84      1700

    accuracy                           0.84      3400
   macro avg       0.84      0.84      0.84      3400
weighted avg       0.84      0.84      0.84      3400



In [38]:
from hpelm import HPELM
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

# Load data
X = ziya_data.drop(columns=['diabetes'])
y = ziya_data['diabetes']

# Split data with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Preprocessing pipeline
num_features = ['age', 'hypertension', 'heart_disease', 'bmi',
                'hbA1c_level', 'blood_glucose_level']
cat_features = ['gender', 'smoking_history']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), cat_features)
    ],
    remainder='passthrough'
)

# Apply preprocessing
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# One-hot encode the target variable for HPELM
encoder = OneHotEncoder(sparse_output=False)
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.values.reshape(-1, 1)) # For consistency if needed later

# Initialize and train HPELM model
inputs = X_train_processed.shape[1]  # Number of features
outputs = y_train_encoded.shape[1]   # Number of classes from one-hot encoding
model = HPELM(inputs, outputs, classification='c')  # Use 'c' for classification

print(f'X: {len(X.columns)}')
# Removed print statements for X_train_processed as they were verbose

# Add neurons to the model (example value, adjust as needed)
model.add_neurons(3000, 'sigm')

# Train model with one-hot encoded target
model.train(X_train_processed, y_train_encoded)

# Predict using the model (predicts one-hot encoded output)
y_pred_encoded = model.predict(X_test_processed)

# Decode predictions back to original labels (0 or 1)
y_pred = encoder.inverse_transform(y_pred_encoded)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


X: 8


/usr/local/lib/python3.12/dist-packages/hpelm/nnets/slfn.py:62: RuntimeWarning: overflow encountered in exp
  self.func["sigm"] = lambda X, W, B: 1 / (1 + np.exp(np.dot(X, W) + B))


processing batch 12/14, eta 0:00:00
Covariance matrix is not full rank; solving with SVD (slow)
This happened because you have duplicated or too many neurons
Accuracy: 0.8582352941176471


In [39]:
from hpelm import HPELM
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, classification_report, precision_recall_curve)

# Load data
X = ziya_data.drop(columns=['diabetes'])
y = ziya_data['diabetes']

# Split data with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Define numerical and categorical features
num_features = ['age', 'hypertension', 'heart_disease', 'bmi',
                'hbA1c_level', 'blood_glucose_level']
cat_features = ['gender', 'smoking_history']

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), cat_features)
    ],
    remainder='passthrough'
)

# Apply preprocessing
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# One-hot encode the target variable
encoder = OneHotEncoder(sparse_output=False)
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.values.reshape(-1, 1))  # For predictions

# Initialize HPELM model
inputs = X_train_processed.shape[1]  # Number of features
outputs = y_train_encoded.shape[1]  # Number of classes
model = HPELM(inputs, outputs, classification='c')  # Use 'c' for classification

# Add neurons to the model
model.add_neurons(3000, 'sigm')  # Add 100 sigmoid neurons, for example

# Train model
model.train(X_train_processed, y_train_encoded)

# Predict using the model
y_pred_encoded = model.predict(X_test_processed)

# Decode predictions back to original labels
y_pred = encoder.inverse_transform(y_pred_encoded)


# Evaluate performance
print("\nEvaluation Metrics")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba[:, 1]):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.12/dist-packages/hpelm/nnets/slfn.py:62: RuntimeWarning: overflow encountered in exp
  self.func["sigm"] = lambda X, W, B: 1 / (1 + np.exp(np.dot(X, W) + B))


processing batch 11/14, eta 0:00:01
Covariance matrix is not full rank; solving with SVD (slow)
This happened because you have duplicated or too many neurons

Evaluation Metrics
Accuracy: 0.8371
Precision: 0.8111
Recall: 0.8788
F1-Score: 0.8436
ROC-AUC: 0.8953

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.80      0.83      1700
           1       0.81      0.88      0.84      1700

    accuracy                           0.84      3400
   macro avg       0.84      0.84      0.84      3400
weighted avg       0.84      0.84      0.84      3400



In [40]:
help(HPELM)

Help on class HPELM in module hpelm.hp_elm:

class HPELM(hpelm.elm.ELM)
 |  HPELM(inputs, outputs, classification='', w=None, batch=1000, accelerator=None, precision='double', norm=None, tprint=5)
 |
 |  Interface for training High-Performance Extreme Learning Machines (HP-ELM).
 |
 |  Args:
 |      inputs (int): dimensionality of input data, or number of data features
 |      outputs (int): dimensionality of output data, or number of classes
 |      classification ('c'/'wc'/'ml', optional): train ELM for classfication ('c') / weighted classification ('wc') /
 |          multi-label classification ('ml'). For weighted classification you can provide weights in `w`. ELM will
 |          compute and use the corresponding classification error instead of Mean Squared Error.
 |      w (vector, optional): weights vector for weighted classification, lenght (`outputs` * 1).
 |      batch (int, optional): batch size for data processing in ELM, reduces memory requirements. Does not work
 |       

In [ ]:
# Logistic Regression

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, classification_report, precision_recall_curve)

# Scale features
scaler = StandardScaler()
X_train_processed = scaler.fit_transform(X_train_processed)
X_test_processed = scaler.transform(X_test_processed)

# Initialize Logistic Regression model
model = LogisticRegression(random_state=42, max_iter=500, solver='saga', C=0.1)

# Train the model
model.fit(X_train_processed, y_train)

# Make predictions
y_pred = model.predict(X_test_processed)

# Evaluate performance
print("\nEvaluation Metrics")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba[:, 1]):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



Evaluation Metrics
Accuracy: 0.8732
Precision: 0.8779
Recall: 0.8671
F1-Score: 0.8724
ROC-AUC: 0.8953

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.88      0.87      1700
           1       0.88      0.87      0.87      1700

    accuracy                           0.87      3400
   macro avg       0.87      0.87      0.87      3400
weighted avg       0.87      0.87      0.87      3400



In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, classification_report, precision_recall_curve)

# Scale features
scaler = StandardScaler()
X_train_processed = scaler.fit_transform(X_train_processed)
X_test_processed = scaler.transform(X_test_processed)

# Initialize Logistic Regression model
model = LogisticRegression(random_state=42, max_iter=500, solver='lbfgs', C=0.1)

# Train the model
model.fit(X_train_processed, y_train)

# Make predictions
y_pred = model.predict(X_test_processed)

# Evaluate performance
print("\nEvaluation Metrics")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba[:, 1]):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



Evaluation Metrics
Accuracy: 0.8732
Precision: 0.8779
Recall: 0.8671
F1-Score: 0.8724
ROC-AUC: 0.8953

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.88      0.87      1700
           1       0.88      0.87      0.87      1700

    accuracy                           0.87      3400
   macro avg       0.87      0.87      0.87      3400
weighted avg       0.87      0.87      0.87      3400



In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {
    'solver': ['lbfgs', 'saga'],
    'C': [0.1, 1, 10],
    'max_iter': [100, 200, 500]
}

grid_search = GridSearchCV(
    estimator=LogisticRegression(),
    param_grid=param_grid,
    scoring='accuracy',  # Use appropriate scoring metric
    cv=5  # Number of folds in cross-validation
)

grid_search.fit(X_train_processed, y_train)
print("Best Parameters:", grid_search.best_params_)


Best Parameters: {'C': 0.1, 'max_iter': 100, 'solver': 'saga'}


In [ ]:
# help(GridSearchCV)

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Scale features
scaler = StandardScaler()
X_train_processed = scaler.fit_transform(X_train_processed)
X_test_processed = scaler.transform(X_test_processed)

# Initialize Logistic Regression model with the tuned parameters
model = LogisticRegression(
    random_state=42,
    max_iter=100,
    solver='saga',
    C=0.1
)

# Train the model
model.fit(X_train_processed, y_train)

# Make predictions
y_pred = model.predict(X_test_processed)

# Evaluate performance
print("\nEvaluation Metrics")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba[:, 1]):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



Evaluation Metrics
Accuracy: 0.8732
Precision: 0.8779
Recall: 0.8671
F1-Score: 0.8724
ROC-AUC: 0.8953

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.88      0.87      1700
           1       0.88      0.87      0.87      1700

    accuracy                           0.87      3400
   macro avg       0.87      0.87      0.87      3400
weighted avg       0.87      0.87      0.87      3400



In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Scale features
scaler = StandardScaler()
X_train_processed = scaler.fit_transform(X_train_processed)
X_test_processed = scaler.transform(X_test_processed)

# Regularized Logistic Regression model
# L1 Regularization (lasso): `penalty='l1'`
# L2 Regularization (ridge): `penalty='l2'`
model = LogisticRegression(
    random_state=42,
    solver='saga',
    C=0.1,  # Regularization strength
    penalty='l1'  # Use L1 or L2
)

# Train the model
model.fit(X_train_processed, y_train)

# Predictions
y_pred = model.predict(X_test_processed)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("Evaluation Metrics")
print("=====================================")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")


Evaluation Metrics
Accuracy: 0.8724
Precision: 0.8777
Recall: 0.8653
F1-Score: 0.8714
ROC-AUC: 0.8724


In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Scale features
scaler = StandardScaler()
X_train_processed = scaler.fit_transform(X_train_processed)
X_test_processed = scaler.transform(X_test_processed)

# Define the parameter grid for regularization strength
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Values to test for regularization strength
    'solver': ['saga'],  # Saga supports both L1 and L2 regularization
    'penalty': ['l1', 'l2']  # Test both L1 (Lasso) and L2 (Ridge)
}

# Grid search for best regularization strength
grid_search = GridSearchCV(
    LogisticRegression(random_state=42, max_iter=500),
    param_grid=param_grid,
    scoring='accuracy',  # You can switch to precision, recall, or other metrics
    cv=5  # 5-fold cross-validation
)

grid_search.fit(X_train_processed, y_train)

# Print the best parameters and corresponding score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Use the best model for evaluation on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_processed)

# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("Evaluation Metrics for Best Model")
print("=====================================")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")


Best Parameters: {'C': 0.01, 'penalty': 'l2', 'solver': 'saga'}
Best Cross-Validation Accuracy: 0.8866911764705883
Evaluation Metrics for Best Model
Accuracy: 0.8732
Precision: 0.8761
Recall: 0.8694
F1-Score: 0.8727
ROC-AUC: 0.8732


In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Scale features
scaler = StandardScaler()
X_train_processed = scaler.fit_transform(X_train_processed)
X_test_processed = scaler.transform(X_test_processed)

# Define the parameter grid for ElasticNet regularization
param_grid = {
    'C': [0.01, 0.1, 1, 10],  # Regularization strengths
    'l1_ratio': [0.2, 0.5, 0.8],  # Mix of L1 and L2 penalties
    'solver': ['saga'],  # Saga supports ElasticNet
    'penalty': ['elasticnet']  # Use ElasticNet penalty
}

# Grid search to find the best ElasticNet parameters
grid_search = GridSearchCV(
    LogisticRegression(random_state=42, max_iter=500),
    param_grid=param_grid,
    scoring='accuracy',  # You can also use precision, recall, etc.
    cv=5  # 5-fold cross-validation
)

grid_search.fit(X_train_processed, y_train)

# Print the best parameters and corresponding score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Use the best model for evaluation on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_processed)

# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("\nEvaluation Metrics for ElasticNet Regularization")
print("====================================================")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")


Best Parameters: {'C': 0.01, 'l1_ratio': 0.5, 'penalty': 'elasticnet', 'solver': 'saga'}
Best Cross-Validation Accuracy: 0.8857352941176471

Evaluation Metrics for ElasticNet Regularization
Accuracy: 0.8712
Precision: 0.8760
Recall: 0.8647
F1-Score: 0.8703
ROC-AUC: 0.8712


In [ ]:
# Random Forest Classifier

In [48]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest model
model = RandomForestClassifier(n_estimators=2500, random_state=42)

# Train the model
model.fit(X_train_processed, y_train)

# Make predictions
y_pred = model.predict(X_test_processed)

# Evaluate performance
print("\nEvaluation Metrics")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba[:, 1]):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



Evaluation Metrics
Accuracy: 0.9021
Precision: 0.8926
Recall: 0.9141
F1-Score: 0.9032
ROC-AUC: 0.8953

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.89      0.90      1700
           1       0.89      0.91      0.90      1700

    accuracy                           0.90      3400
   macro avg       0.90      0.90      0.90      3400
weighted avg       0.90      0.90      0.90      3400



In [ ]:
# Support Vector Machine(SVM)

In [49]:
from sklearn.svm import SVC

# Initialize SVM model
model = SVC(kernel='rbf', random_state=42)

# Train the model
model.fit(X_train_processed, y_train)

# Make predictions
y_pred = model.predict(X_test_processed)

# Evaluate performance
print("\nEvaluation Metrics")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba[:, 1]):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



Evaluation Metrics
Accuracy: 0.8803
Precision: 0.8761
Recall: 0.8859
F1-Score: 0.8810
ROC-AUC: 0.8953

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.87      0.88      1700
           1       0.88      0.89      0.88      1700

    accuracy                           0.88      3400
   macro avg       0.88      0.88      0.88      3400
weighted avg       0.88      0.88      0.88      3400



In [ ]:
# K-Nearest Neighbourh

In [50]:
from sklearn.neighbors import KNeighborsClassifier

# Initialize KNN model
model = KNeighborsClassifier(n_neighbors=5)

# Train the model
model.fit(X_train_processed, y_train)

# Make predictions
y_pred = model.predict(X_test_processed)

# Evaluate performance
print("\nEvaluation Metrics")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba[:, 1]):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



Evaluation Metrics
Accuracy: 0.8679
Precision: 0.8564
Recall: 0.8841
F1-Score: 0.8700
ROC-AUC: 0.8953

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.85      0.87      1700
           1       0.86      0.88      0.87      1700

    accuracy                           0.87      3400
   macro avg       0.87      0.87      0.87      3400
weighted avg       0.87      0.87      0.87      3400



# Voting Ensemble

In [51]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Scale features
scaler = StandardScaler()
X_train_processed = scaler.fit_transform(X_train_processed)
X_test_processed = scaler.transform(X_test_processed)

# Initialize individual models
log_reg = LogisticRegression(random_state=42, C=0.01, penalty='l2', solver='saga', max_iter=500)  # Best-tuned LR model
rf_clf = RandomForestClassifier(random_state=42, n_estimators=100)  # Random Forest
svm_clf = SVC(probability=True, random_state=42)  # Support Vector Machine with probability for soft voting

# Create the VotingClassifier (use 'soft' or 'hard' voting)
voting_clf = VotingClassifier(
    estimators=[('lr', log_reg), ('rf', rf_clf), ('svm', svm_clf)],
    voting='soft'  # For soft voting based on probabilities
)

# Train the ensemble model
voting_clf.fit(X_train_processed, y_train)

# Make predictions
y_pred = voting_clf.predict(X_test_processed)

# Evaluate the ensemble model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("Voting Ensemble Metrics")
print("=================================")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")


Voting Ensemble Metrics
Accuracy: 0.8888
Precision: 0.8865
Recall: 0.8918
F1-Score: 0.8891
ROC-AUC: 0.8888


# Stacking Ensemble

In [52]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Scale features
scaler = StandardScaler()
X_train_processed = scaler.fit_transform(X_train_processed)
X_test_processed = scaler.transform(X_test_processed)

# Define base models
base_models = [
    ('rf', RandomForestClassifier(random_state=42, n_estimators=100)),  # Random Forest
    ('svm', SVC(probability=True, random_state=42))  # Support Vector Machine with probability output
]

# Define the meta-model (Logistic Regression in this case)
meta_model = LogisticRegression(random_state=42, solver='saga', C=0.1, penalty='l2')

# Create the StackingClassifier
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5  # Cross-validation during training of the meta-model
)

# Train the Stacking Ensemble
stacking_clf.fit(X_train_processed, y_train)

# Make predictions
y_pred = stacking_clf.predict(X_test_processed)

# Evaluate the Stacking Ensemble
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("Stacking Ensemble Metrics")
print("=================================")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")


Stacking Ensemble Metrics
Accuracy: 0.8974
Precision: 0.8907
Recall: 0.9059
F1-Score: 0.8982
ROC-AUC: 0.8974


## Tuning the Base Models and Meta-Model

### Step 1: Tune the Base Models

In [53]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Tuning Random Forest
rf_param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees
    'max_depth': [10, 20, None],     # Maximum depth of trees
    'min_samples_split': [2, 5, 10]  # Minimum samples to split a node
}

rf_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid=rf_param_grid,
    scoring='accuracy',
    cv=5
)
rf_search.fit(X_train_processed, y_train)
best_rf = rf_search.best_estimator_
print("Best Random Forest Parameters:", rf_search.best_params_)

# Tuning SVM
svm_param_grid = {
    'C': [0.1, 1, 10],           # Regularization parameter
    'kernel': ['linear', 'rbf'],  # Kernel type
    'gamma': ['scale', 'auto']    # Kernel coefficient
}

svm_search = GridSearchCV(
    SVC(probability=True, random_state=42),
    param_grid=svm_param_grid,
    scoring='accuracy',
    cv=5
)
svm_search.fit(X_train_processed, y_train)
best_svm = svm_search.best_estimator_
print("Best SVM Parameters:", svm_search.best_params_)


Best Random Forest Parameters: {'max_depth': None, 'min_samples_split': 10, 'n_estimators': 200}
Best SVM Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}


### Step 2: Tune the Meta-Model

In [54]:
from sklearn.linear_model import LogisticRegression

# Tuning Logistic Regression (meta-model)
lr_param_grid = {
    'C': [0.01, 0.1, 1, 10],  # Regularization strength
    'solver': ['saga'],       # Solver for ElasticNet
    'penalty': ['l1', 'l2']   # Regularization type
}

lr_search = GridSearchCV(
    LogisticRegression(random_state=42, max_iter=500),
    param_grid=lr_param_grid,
    scoring='accuracy',
    cv=5
)
lr_search.fit(X_train_processed, y_train)
best_meta_model = lr_search.best_estimator_
print("Best Logistic Regression Parameters:", lr_search.best_params_)


Best Logistic Regression Parameters: {'C': 0.01, 'penalty': 'l2', 'solver': 'saga'}


### Step 3: Integrate Tuned Models into Stacking Ensemble
Finally, use the tuned base models (best_rf, best_svm) and the meta-model (best_meta_model) in the stacking classifier:

In [55]:
from sklearn.ensemble import StackingClassifier

# Define tuned models
base_models = [
    ('rf', best_rf),
    ('svm', best_svm)
]

# Stacking classifier with tuned models
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=best_meta_model,
    cv=5
)

# Train the optimized Stacking Ensemble
stacking_clf.fit(X_train_processed, y_train)

# Evaluate the ensemble
y_pred = stacking_clf.predict(X_test_processed)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("\nOptimized Stacking Ensemble Metrics")
print("======================================")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")



Optimized Stacking Ensemble Metrics
Accuracy: 0.9000
Precision: 0.8981
Recall: 0.9024
F1-Score: 0.9002
ROC-AUC: 0.9000


# Bagging Ensemble Model
Bagging, short for Bootstrap Aggregating

In [56]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Scale features
scaler = StandardScaler()
X_train_processed = scaler.fit_transform(X_train_processed)
X_test_processed = scaler.transform(X_test_processed)

# Initialize base model (Decision Tree in this case)
base_model = DecisionTreeClassifier(random_state=42)

# Create BaggingClassifier
bagging_clf = BaggingClassifier(
    estimator=base_model,  # Correct keyword for base model
    n_estimators=50,       # Number of base models
    max_samples=0.8,       # Fraction of samples used for training each base model
    max_features=0.8,      # Fraction of features used for training each base model
    bootstrap=True,        # Enable bootstrapping of samples
    random_state=42
)

# Train the Bagging model
bagging_clf.fit(X_train_processed, y_train)

# Make predictions
y_pred = bagging_clf.predict(X_test_processed)

# Evaluate the Bagging model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("\nBagging Ensemble Metrics")
print("=======================================")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")



Bagging Ensemble Metrics
Accuracy: 0.8976
Precision: 0.8890
Recall: 0.9088
F1-Score: 0.8988
ROC-AUC: 0.8976


## hyperparameter Tuning for Bagging

In [57]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Scale features
scaler = StandardScaler()
X_train_processed = scaler.fit_transform(X_train_processed)
X_test_processed = scaler.transform(X_test_processed)

# Define the Bagging model with a Decision Tree as the base estimator
base_model = DecisionTreeClassifier(random_state=42)

# Parameter grid for Bagging
param_grid = {
    'n_estimators': [10, 50, 100],  # Number of estimators
    'max_samples': [0.5, 0.8, 1.0],  # Fraction of training samples for each base estimator
    'max_features': [0.5, 0.8, 1.0],  # Fraction of features for each base estimator
    'estimator__max_depth': [3, 5, None],  # Max depth of the base Decision Tree
    'estimator__min_samples_split': [2, 5, 10]  # Min samples to split a node
}

# GridSearchCV for BaggingClassifier
bagging_clf = BaggingClassifier(
    estimator=base_model,
    bootstrap=True,  # Bootstrap samples
    random_state=42
)

grid_search = GridSearchCV(
    estimator=bagging_clf,
    param_grid=param_grid,
    scoring='accuracy',  # Evaluate using accuracy
    cv=5  # 5-fold cross-validation
)

# Fit the grid search on the training data
grid_search.fit(X_train_processed, y_train)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Evaluate the best model on the test set
best_bagging_clf = grid_search.best_estimator_
y_pred = best_bagging_clf.predict(X_test_processed)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("\nOptimized Bagging Ensemble Metrics")
print("=======================================")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")


Best Parameters: {'estimator__max_depth': None, 'estimator__min_samples_split': 10, 'max_features': 0.5, 'max_samples': 0.8, 'n_estimators': 100}
Best Cross-Validation Accuracy: 0.9098529411764705

Optimized Bagging Ensemble Metrics
Accuracy: 0.9074
Precision: 0.8924
Recall: 0.9265
F1-Score: 0.9091
ROC-AUC: 0.9074
